#Tool-calling Agent

This is an auto-generated notebook created by an AI Playground export.

This notebook uses [Mosaic AI Agent Framework](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/build-genai-apps) to recreate your agent from the AI Playground. It  demonstrates how to develop, manually test, evaluate, log, and deploy a tool-calling agent in LangGraph.

The agent code implements [MLflow's ChatAgent](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent) interface, a Databricks-recommended open-source standard that simplifies authoring multi-turn conversational agents, and is fully compatible with Mosaic AI agent framework functionality.

 **_NOTE:_**  This notebook uses LangChain, but AI Agent Framework is compatible with any agent authoring framework, including LlamaIndex or pure Python agents written with the OpenAI SDK.

## Prerequisites

- Address all `TODO`s in this notebook.

In [0]:
%pip install -U -qqqq mlflow langchain langgraph==0.3.4 databricks-langchain pydantic databricks-agents unitycatalog-langchain[databricks] uv
dbutils.library.restartPython()

## Define the agent in code
Below we define our agent code in a single cell, enabling us to easily write it to a local Python file for subsequent logging and deployment using the `%%writefile` magic command.

For more examples of tools to add to your agent, see [docs](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/agent-tool).

In [0]:
%%writefile agent.py
from typing import Any, Generator, Optional, Sequence, Union
from langchain.tools import Tool
import mlflow
from databricks_langchain import ChatDatabricks
from databricks_langchain.uc_ai import (
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)
from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_node import ToolNode
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
import re

mlflow.langchain.autolog()

client = DatabricksFunctionClient()
set_uc_function_client(client)

############################################
# Define your LLM endpoint and system prompt
############################################
#LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"
# LLM_ENDPOINT_NAME = "databricks-claude-sonnet-4-5"
#LLM_ENDPOINT_NAME = "llama_v3_3_70b_instruct_pt"
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

system_prompt = f"""                
                You are a product recommendation agent for conversational commerce.

TOOLS:
- get_smart_product_recommendations: Get personalized recommendations (use customer name, email, search query, brand, min/max price)
- product_lookup_by_description: Look up specific products by description
- get_random_customer_info: Get a random customer if none provided
- get_customer_interactions: Get customer history

RULES:
1. Call tools ONCE per query, then respond immediately
2. For customer queries with name/email: call get_smart_product_recommendations directly
3. Format response: "Based on your preferences, I recommend:\n\n**Category - Subcategory:**\n1. Product Name - $Price (Rating) - Brief description"
4. Group products by category in response
5. Honor price/brand filters from user query
6. If no customer context: call get_random_customer_info first
7. Use this example to form the tool call. If a search query says - "Show me cloths under 250 $ from Zara" from a customer named "Dawn Smith" and customer email "griffinheidi@example.org" you will call the get_smart_product_recommendations () tool with the following parameters -
        (
            "input_customer_name": "Dawn Smith",
            "input_customer_email": "griffinheidi@example.org",
            "input_product_description": "cloths",
            "input_brand": "Zara",
            "input_max_price": 250
        )

RESPONSE FORMAT EXAMPLE:
Based on your preferences, I recommend the following products for your camping trip:

**Sports & Outdoors - Camping:**
1. Under Armour Camping West - $105.01 (4.7/5) - Water-resistant with multiple compartments

Would you like more details on any item?

Keep responses concise and helpful.  
    """

tools = []

# You can use UDFs in Unity Catalog as agent tools
uc_tool_names = [
    "retail_consumer_goods.conversational_commerce_search.get_smart_product_recommendations",
    "retail_consumer_goods.conversational_commerce_search.product_lookup_by_description",
    "retail_consumer_goods.conversational_commerce_search.validate_customer_by_name",
    "retail_consumer_goods.conversational_commerce_search.get_customer_interactions",
    "retail_consumer_goods.conversational_commerce_search.get_random_customer_info",
]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools.extend(uc_toolkit.tools)


def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: Union[Sequence[BaseTool], ToolNode],
    system_prompt: Optional[str] = None,
) -> CompiledGraph:
    model = model.bind_tools(tools)

    # Define the function that determines which node to go to
    def should_continue(state: ChatAgentState):
        messages = state["messages"]
        last_message = messages[-1]
        # If there are function calls, continue. else, end
        if last_message.get("tool_calls"):
            return "continue"
        else:
            return "end"

    if system_prompt:
        preprocessor = RunnableLambda(
            lambda state: [{"role": "system", "content": system_prompt}]
            + state["messages"]
        )
    else:
        preprocessor = RunnableLambda(lambda state: state["messages"])
    model_runnable = preprocessor | model

    def call_model(
        state: ChatAgentState,
        config: RunnableConfig,
    ):
        response = model_runnable.invoke(state, config)

        return {"messages": [response]}

    workflow = StateGraph(ChatAgentState)

    workflow.add_node("agent", RunnableLambda(call_model))
    workflow.add_node("tools", ChatAgentToolNode(tools))

    workflow.set_entry_point("agent")
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "continue": "tools",
            "end": END,
        },
    )
    workflow.add_edge("tools", "agent")

    return workflow.compile()


class LangGraphChatAgent(ChatAgent):
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        request = {"messages": self._convert_messages_to_dict(messages)}

        messages = []
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                messages.extend(
                    ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
                )
        return ChatAgentResponse(messages=messages)

    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        request = {"messages": self._convert_messages_to_dict(messages)}
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                yield from (
                    ChatAgentChunk(**{"delta": msg}) for msg in node_data["messages"]
                )


# Create the agent object, and specify it as the agent object to use when
# loading the agent back for inference via mlflow.models.set_model()
agent = create_tool_calling_agent(llm, tools, system_prompt)
AGENT = LangGraphChatAgent(agent)
mlflow.models.set_model(AGENT)

## Test the agent

Interact with the agent to test its output. Since this notebook called `mlflow.langchain.autolog()` you can view the trace for each step the agent takes.

Replace this placeholder input with an appropriate domain-specific example for your agent.

In [0]:
dbutils.library.restartPython()

In [0]:
from IPython.display import Image
from agent import agent

display(Image(agent.get_graph().draw_mermaid_png()))


In [ ]:
from agent import AGENT

# Test the agent with image integration
print("Testing agent integration - 0...")
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "Customer Name: Maria Shaw, Customer email: ian49@example.org. Find me home decor items under $150."}]}
):
    print(event, "-----------\n")


In [ ]:
from agent import AGENT

# Test the agent with image integration
print("Testing agent integration - 1...")
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "Customer Name: Dawn Smith, Customer email: griffinheidi@example.org. Show me cloths under 250 $ from Zara."}]}
):
    print(event, "-----------\n")


In [0]:
from agent import AGENT

# Test the agent with image integration
print("Testing agent integration - 2...")
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "Customer Name: Joanne Strong, Customer email: bsmith@example.net. I am going for a camping trip where it's going to rain. I want to be prepared. I would also like to bring some books for leisure reading and plan to do lots of hiking and other activities. Each item should not be more than 500 $."}]}
):
    print(event, "-----------\n")


In [0]:
from agent import AGENT

# Test the agent with image integration
print("Testing agent integration - 3...")
for event in AGENT.predict_stream(
    {"messages": [
        {
            "role": "user", 
            "content": "Customer Name: Ronald Jackson, Customer email: ureed@example.com. I am moving to a new apartment and need to buy all the necessary items for my new apartment. I do not want them to be too expensive but durable. What do you suggest?"
        }
    ],
     "max_tokens": 4000
     }
):
    print(event, "-----------\n")

In [0]:
from agent import AGENT

# Test the agent with image integration
print("Testing agent integration - 4...")
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "Customer Name: Susan Martin, Customer email: tcontreras@example.net. I am looking for home improvement products less than 2500 $"}]}
):
    print(event, "-----------\n")


In [0]:
from agent import AGENT

# Test the agent with image integration
print("Testing agent integration - 5...")
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "Customer Name: Jamie Fry, Customer email: tony60@example.com. I work in a really loud place and need to take a lot of meetings. Looking for some high end headphones that can work well in this situation."}]}
):
    print(event, "-----------\n")
 

In [0]:
from agent import AGENT

# Test the agent with image integration
print("Testing agent integration - 6...")
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "Show me some options for gaming laptop."}]}
):
    print(event, "-----------\n")


In [0]:
from agent import AGENT

# Test the agent with image integration
print("Testing agent integration - 7...")
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "Recommend some fictional books below 20 $"}]}
):
    print(event, "-----------\n")

### Log the `agent` as an MLflow model
Determine Databricks resources to specify for automatic auth passthrough at deployment time
- **TODO**: If your Unity Catalog tool queries a [vector search index](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/unstructured-retrieval-tools) or leverages [external functions](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/external-connection-tools), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See [docs](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#specify-resources-for-automatic-authentication-passthrough) for more details.

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import mlflow
from agent import tools, LLM_ENDPOINT_NAME
from databricks_langchain import VectorSearchRetrieverTool, ChatDatabricks
from mlflow.models.resources import (
  DatabricksVectorSearchIndex,
  DatabricksServingEndpoint,
  DatabricksFunction,
  DatabricksTable
)
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

# TODO: Manually include underlying resources if needed. See the TODO in the markdown above for more information.
resources = [
             DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
             DatabricksVectorSearchIndex(index_name="retail_consumer_goods.conversational_commerce_search.product_index"),
             DatabricksVectorSearchIndex(index_name="retail_consumer_goods.conversational_commerce_search.user_index"),
             DatabricksVectorSearchIndex(index_name="retail_consumer_goods.conversational_commerce_search.product_desc_index"),
             DatabricksFunction(function_name="retail_consumer_goods.conversational_commerce_search.get_smart_product_recommendations"),
             DatabricksFunction(function_name="retail_consumer_goods.conversational_commerce_search.product_lookup_by_description"),
             DatabricksFunction(function_name="retail_consumer_goods.conversational_commerce_search.validate_customer_by_name"),
             DatabricksFunction(function_name="retail_consumer_goods.conversational_commerce_search.get_customer_interactions"),
             DatabricksFunction(function_name="retail_consumer_goods.conversational_commerce_search.get_random_customer_info"),
             DatabricksTable(table_name="retail_consumer_goods.conversational_commerce_search.customers"),
             DatabricksTable(table_name="retail_consumer_goods.conversational_commerce_search.interactions"),
             DatabricksTable(table_name="retail_consumer_goods.conversational_commerce_search.product_features"),
             DatabricksTable(table_name="retail_consumer_goods.conversational_commerce_search.products"),
             DatabricksTable(table_name="retail_consumer_goods.conversational_commerce_search.user_features")
             ]

for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "I am Susan Martin. My email is tcontreras@example.net. I am looking for non fiction books below 20 $"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        pip_requirements=[
            "mlflow",
            "langchain",
            "langgraph==0.3.4",
            "databricks-langchain==0.4.1",
            "unitycatalog-langchain[databricks]",
            "pydantic",
        ],
        resources=resources,
    )

## Register the model to Unity Catalog

Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.errors import NotFound

In [0]:
mlflow.set_registry_uri("databricks-uc")

catalog = "retail_consumer_goods"
schema = "conversational_commerce_search"
model_name = "convo_comm_search_model"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

### ONLY RUN the following cell if you want to DELETE the model including all its versions

In [0]:
# w = WorkspaceClient()

# try:
#     model_info = w.registered_models.get(full_name=UC_MODEL_NAME)
#     print(f"✅ Model '{UC_MODEL_NAME}' exists")
#     versions = w.model_versions.list(UC_MODEL_NAME)

#     for v in versions:
#         print(f"Deleting version {v.version} of {UC_MODEL_NAME} ...")
#         w.model_versions.delete(
#             full_name=UC_MODEL_NAME,
#             version=v.version
#         )

#     print(f"Deleting model {UC_MODEL_NAME} ...")
#     w.registered_models.delete(
#         full_name=UC_MODEL_NAME
#     )
#     print(f"✅ All versions and model '{UC_MODEL_NAME}' deleted successfully.")
# except NotFound:
#     print(f"❌ Model '{UC_MODEL_NAME}' does not exist")



In [0]:
# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

## Deploy the agent

In [0]:
from databricks import agents
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "playground"})

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See [docs](https://learn.microsoft.com/azure/databricks/generative-ai/deploy-agent) for details